In [4]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [6]:
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("aboutGoogle").load_data()

Ignoring wrong pointing object 0 0 (offset 0)


In [7]:
import sys
print(sys.executable)
!pip list

c:\Users\poora\OneDrive\Desktop\projects\RAG\env\Scripts\python.exe
Package                                 Version
--------------------------------------- -----------
aiohttp                                 3.9.3
aiosignal                               1.3.1
annotated-types                         0.6.0
anyio                                   4.3.0
asttokens                               2.4.1
async-timeout                           4.0.3
attrs                                   23.2.0
beautifulsoup4                          4.12.3
certifi                                 2024.2.2
charset-normalizer                      3.3.2
click                                   8.1.7
colorama                                0.4.6
comm                                    0.2.2
dataclasses-json                        0.6.4
debugpy                                 1.8.1
decorator                               5.1.1
Deprecated                              1.2.14
dirtyjson                               1.0.


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
documents

[Document(id_='1a094bcd-051e-4290-8c38-c57dedf6a89c', embedding=None, metadata={'page_label': '1', 'file_name': 'Employee Handbook 2013-14.pdf', 'file_path': 'aboutGoogle\\Employee Handbook 2013-14.pdf', 'file_type': 'application/pdf', 'file_size': 677991, 'creation_date': '2024-04-09', 'last_modified_date': '2024-04-09', 'last_accessed_date': '2024-04-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' \n Slocum ISD  \n Employee Handbook  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n2013-2014 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nThis book is provided for educational purposes only and contains information to facilitate a general understanding of \nthe law. It is not an exhaustive treatment of the law on thes e subjects no

In [10]:
index = VectorStoreIndex.from_documents(documents,show_progress=True)

c:\Users\poora\OneDrive\Desktop\projects\RAG\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 152/152 [00:03<00:00, 42.01it/s]


In [11]:
index

In [12]:
query_engine = index.as_query_engine()

In [13]:
query_engine

In [11]:
from llama_index.core import Settings

In [10]:
from llama_index.legacy import  StorageContext

In [9]:
from llama_index.legacy import load_index_from_storage

In [12]:
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("aboutGoogle").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
    
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)
    
query_engine = index.as_query_engine() # print single response
response = query_engine.query("how does google promote diversity inclusion?")
print(response)

Google promotes diversity and inclusion through various initiatives such as providing mental health services, combatting hate speech online, supporting career growth for people with disabilities, launching programs like gReach for individuals with disabilities, creating spaces for LGBTQ+ communities, investing in career success for future generations, focusing on equity in the hiring process, offering inclusive hiring training, and partnering with organizations like Color in Tech to connect with underrepresented talent.


In [20]:
# top k retrieved responses

from llama_index.legacy.retrievers import VectorIndexRetriever
from llama_index.legacy.query_engine import RetrieverQueryEngine
from llama_index.legacy.postprocessor import SimilarityPostprocessor

retriever = VectorIndexRetriever(index=index,similarity_top_k=5)
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.80)
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[postprocessor])

response = query_engine.query("what does google do?")
from llama_index.legacy.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: Google offers a wide range of services and products,
including a search engine, Gmail, maps, personal blogging, web-based
word processing, YouTube, and enterprise versions of their consumer
products for businesses, organizations, and government entities. They
generate revenue primarily through their AdWords advertising program,
which allows businesses to place sponsored links alongside search
results. Google also provides tools like Google Apps for cloud
computing, which help users collaborate more effectively and save
money. Additionally, Google.org, a philanthropic arm of Google, uses
information and technology to address global challenges and support
initiatives related to predicting and preventing emerging threats,
providing information to empower communities, and promoting government
openness and transparency.
______________________________________________________________________
Source Node 1/5
Node ID: c28e05d7-0888-44b1-ab98-02eddf1b2532
Similarity: 0.8659906330

In [24]:
chat_engine = index.as_chat_engine()
streaming_response = chat_engine.stream_chat("can you tell me what does google do?")
for token in streaming_response.response_gen:
    print(token, end="")

Google offers a variety of services beyond being a search engine, such as Gmail, maps, personal blogging, and web-based word processing products. It also owns YouTube. Google generates revenue primarily through its AdWords advertising program and AdSense advertising program. Additionally, Google provides enterprise versions of its consumer products for businesses, organizations, and government entities. Google.org, a philanthropic arm of Google, uses information and technology to address global challenges and works on projects leveraging Google's strengths in technology and information.